In [2]:
class Particle:
    """
    Represents a single particle in the PSO swarm

    Each particle represents one complete ANN (all weights and biases)
    and tracks its position, velocity, and best-found solution
    """

    def __init__(self, dimension: int, bounds: Tuple[float, float]):
        """
        Initialize a particle with random position and velocity

        Corresponds to Lines 1-2 of Algorithm 39

        Args:
            dimension: Number of parameters to optimize (ANN parameter count)
            bounds: Tuple of (min_value, max_value) for initialization
        """
        self.dimension = dimension
        self.bounds = bounds

        # Line 1: Initialize position randomly within bounds
        self.position = np.random.uniform(bounds[0], bounds[1], dimension)

        # Line 2: Initialize velocity randomly
        # Velocity range is half the position range
        velocity_range = (bounds[1] - bounds[0]) / 2
        self.velocity = np.random.uniform(-velocity_range, velocity_range, dimension)

        # Personal best: best position this particle has found
        self.best_position = self.position.copy()
        self.best_fitness = float('inf')  # Using minimization

        # Current fitness value
        self.fitness = float('inf')

        # Informants: subset of particles that influence this particle
        # Will be set by PSO class
        self.informants = []

    def update_velocity(self, informant_best_position: np.ndarray,
                       phi1: float, phi2: float, phi3: float):
        """
        Update particle velocity using PSO equation

        Corresponds to Lines 11-12 of Algorithm 39

        Velocity update equation:
        v = phi3 * v + phi1 * r1 * (p_best - x) + phi2 * r2 * (informant_best - x)

        Args:
            informant_best_position: Best position among particle's informants
            phi1: Cognitive coefficient (attraction to personal best)
            phi2: Social coefficient (attraction to informant best)
            phi3: Inertia weight (momentum)
        """
        # Generate random vectors for stochastic components
        # Each dimension gets its own random value
        r1 = np.random.random(self.dimension)
        r2 = np.random.random(self.dimension)

        # Line 11: Cognitive component - attraction to personal best
        cognitive = phi1 * r1 * (self.best_position - self.position)

        # Line 12: Social component - attraction to informant best
        social = phi2 * r2 * (informant_best_position - self.position)

        # Line 12: Update velocity with inertia (momentum)
        self.velocity = phi3 * self.velocity + cognitive + social

    def update_position(self):
        """
        Update particle position based on current velocity

        Corresponds to Line 13 of Algorithm 39

        Applies boundary handling to keep position within valid bounds:
        - Clamps position to bounds
        - Reverses velocity component when hitting boundary (bounce-back)
        """
        # Line 13: Update position
        self.position = self.position + self.velocity

        # Boundary handling: ensure position stays within bounds
        for i in range(self.dimension):
            if self.position[i] < self.bounds[0]:
                # Hit lower bound: clamp position and reverse velocity
                self.position[i] = self.bounds[0]
                self.velocity[i] = -self.velocity[i]
            elif self.position[i] > self.bounds[1]:
                # Hit upper bound: clamp position and reverse velocity
                self.position[i] = self.bounds[1]
                self.velocity[i] = -self.velocity[i]

# ============================================================================
# PARTICLE SWARM OPTIMIZATION (PSO) CLASS
# ============================================================================

class PSO:
    """
    Particle Swarm Optimization Algorithm

    Implements Algorithm 39 from "Essentials of Metaheuristics" by Sean Luke
    Uses informant topology (not global best)
    """

    def __init__(self, swarm_size: int, dimension: int, bounds: Tuple[float, float],
                 num_informants: int = 3, phi1: float = 2.05, phi2: float = 2.05,
                 phi3: float = 0.729):
        """
        Initialize PSO with specified parameters

        Args:
            swarm_size: Number of particles in the swarm
            dimension: Dimensionality of search space (number of ANN parameters)
            bounds: (min, max) tuple for parameter initialization and boundaries
            num_informants: Number of informants per particle (K in Algorithm 39)
            phi1: Cognitive acceleration coefficient
            phi2: Social acceleration coefficient
            phi3: Inertia weight (constriction coefficient)
        """
        self.swarm_size = swarm_size
        self.dimension = dimension
        self.bounds = bounds
        self.num_informants = num_informants

        # PSO coefficients
        self.phi1 = phi1  # Cognitive (personal best attraction)
        self.phi2 = phi2  # Social (informant best attraction)
        self.phi3 = phi3  # Inertia/constriction

        # Lines 1-2: Create swarm of particles
        self.particles = [Particle(dimension, bounds) for _ in range(swarm_size)]

        # Line 3: Assign informants to each particle
        self._assign_informants()

        # Track global best across entire swarm
        self.global_best_position = None
        self.global_best_fitness = float('inf')

        # Track fitness evolution over time
        self.fitness_history = []

    def _assign_informants(self):
        """
        Assign K random informants to each particle

        Corresponds to Line 3 of Algorithm 39

        Each particle is influenced by K randomly selected other particles
        (not including itself)
        """
        for particle in self.particles:
            # Get list of all other particles (excluding current particle)
            other_particles = [p for p in self.particles if p != particle]

            # Randomly select K informants from other particles
            k = min(self.num_informants, len(other_particles))
            particle.informants = random.sample(other_particles, k)

    def _get_informant_best(self, particle: Particle) -> np.ndarray:
        """
        Find the best position among a particle's informants

        Corresponds to Line 10 of Algorithm 39

        Args:
            particle: The particle whose informant best to find

        Returns:
            Position vector of best-performing informant
        """
        # Find informant with lowest fitness (best performance)
        best_informant = min(particle.informants, key=lambda p: p.best_fitness)
        return best_informant.best_position

    def optimize(self, fitness_function: Callable, max_iterations: int) -> Tuple[np.ndarray, float]:
        """
        Run PSO optimization

        Implements Lines 4-15 of Algorithm 39

        Args:
            fitness_function: Function to evaluate particle fitness
                            Takes position vector, returns fitness value (lower is better)
            max_iterations: Number of iterations to run

        Returns:
            Tuple of (best_position, best_fitness)
        """
        # Lines 4-8: Evaluate initial swarm
        for particle in self.particles:
            # Line 5: Evaluate particle fitness
            particle.fitness = fitness_function(particle.position)

            # Lines 6-8: Update personal best if this is better
            if particle.fitness < particle.best_fitness:
                particle.best_fitness = particle.fitness
                particle.best_position = particle.position.copy()

            # Update global best (for tracking purposes)
            if particle.fitness < self.global_best_fitness:
                self.global_best_fitness = particle.fitness
                self.global_best_position = particle.position.copy()

        # Record initial best fitness
        self.fitness_history.append(self.global_best_fitness)

        # Line 9: Main optimization loop - repeat for specified iterations
        for iteration in range(max_iterations):

            # Process each particle in the swarm
            for particle in self.particles:
                # Line 10: Get best position among this particle's informants
                informant_best = self._get_informant_best(particle)

                # Lines 11-12: Update velocity based on cognitive and social components
                particle.update_velocity(informant_best, self.phi1, self.phi2, self.phi3)

                # Line 13: Update position based on velocity
                particle.update_position()

                # Line 14: Evaluate fitness at new position
                particle.fitness = fitness_function(particle.position)

                # Line 15: Update personal best if new position is better
                if particle.fitness < particle.best_fitness:
                    particle.best_fitness = particle.fitness
                    particle.best_position = particle.position.copy()

                # Update global best (for tracking)
                if particle.fitness < self.global_best_fitness:
                    self.global_best_fitness = particle.fitness
                    self.global_best_position = particle.position.copy()

            # Record best fitness this iteration
            self.fitness_history.append(self.global_best_fitness)

            # Print progress every 10 iterations
            if (iteration + 1) % 10 == 0 or iteration == 0:
                print(f"  Iteration {iteration + 1}/{max_iterations}, "
                      f"Best Fitness (MAE): {self.global_best_fitness:.4f}")

        return self.global_best_position, self.global_best_fitness
